In [1]:
from IPython import get_ipython
ip = get_ipython()
ip.run_line_magic("reload_ext", "autoreload")  # these will enable module autoreloading
ip.run_line_magic("autoreload", "2")

### Load in Exports

In [2]:
import os
import src.BrokerageExportProviders.Brokerages.IBKR.IbkrBrokerageExportProvider as ibrk
import src.TaxAuthorityProvider.TaxAuthorities.Slovenia.SlovenianTaxAuthorityProvider as sir
import src.ConfigurationProvider.Configuration as conf
import src.TaxAuthorityProvider.Utils.GenericUtilities as gu
import arrow

rootOfProject = os.getcwd()

reportsDirectory = os.path.join(rootOfProject, "imports")
generatedDirectory = os.path.join(rootOfProject, "exports")
configDirectory = os.path.join(rootOfProject, "config")

configProvider = conf.ConfigurationProvider(configDirectory)
taxPayerInfo = configProvider.getConfig()

ibkrProvider = ibrk.IbkrBrokerageExportProvider()
brokerExports = ibkrProvider.getListOfReportsAvailableForBroker(reportsDirectory)
print(brokerExports)



def loadFileAndExtractLines(file: str):  
    transactions = ibkrProvider.getBrokerEventsForReport(file)
    return transactions

brokerReports = list(map(loadFileAndExtractLines, brokerExports))
mergedReports = ibkrProvider.mergeBrokerEvents(brokerReports)

['/mnt/extra/Projects/IB-tax-calculator/imports/Tax_Relevant_Export_2023.xml', '/mnt/extra/Projects/IB-tax-calculator/imports/Tax_Relevant_Export_2022.xml']


### Dividends Reports

In [7]:
reportconfig = sir.EDavkiReportConfig(fromDate=arrow.get("2022"), toDate=arrow.get("2023"), ReportType=sir.EDavkiDocumentWorkflowType.ORIGINAL)

dividendReport = sir.EDavkiDividendReport(reportConfig=reportconfig, taxPayerInfo=taxPayerInfo)
envelope = dividendReport.createReportEnvelope()

convertedCommonFormat = ibrkMapper.getGenericDividendLineFromIBRKCashTransactions(mergedReports.cashTransactions)

csvReport = dividendReport.generateDataFrameReport(convertedCommonFormat)
csvReport.to_csv(os.path.join(generatedDirectory, 'export.csv'), index=False)

xmlReport = dividendReport.generateXmlReport(convertedCommonFormat, envelope)


etree.ElementTree(xmlReport).write(os.path.join(generatedDirectory, "Doh_Div_3.xml"), xml_declaration=True, encoding='utf-8', pretty_print=True)



### Stock Trades

In [3]:
reportconfig = sir.EDavkiReportConfig(fromDate=arrow.get("2022"), toDate=arrow.get("2023"), ReportType=sir.EDavkiDocumentWorkflowType.ORIGINAL)
convertedCommonFormat = ibkrProvider.transformBrokerEventsToBrokerAgnosticEvents(mergedReports)
gUtils = gu.GenericUtilities()


converted = gUtils.generateGenericGroupings(convertedCommonFormat)
print(converted)

# print(convertedCommonFormat)
tradeReport = sir.EDavkiTradesReport(reportConfig=reportconfig, taxPayerInfo=taxPayerInfo)
envelope = tradeReport.createReportEnvelope()


tradeCsv = tradeReport.generateDataFrameReport(converted)
tradeCsv.to_csv(os.path.join(generatedDirectory, 'export-trades.csv'), index=False)



xmlReport = tradeReport.generateXmlReport(converted, envelope)


etree.ElementTree(xmlReport).write(os.path.join(generatedDirectory, "Doh_KDVP_9.xml"), xml_declaration=True, encoding='utf-8', pretty_print=True)


Processing stock lot (ID: 497227221)
Matched Buy with trade (ID: 497227221, DateTime: 2023-05-08T03:02:11-04:00)
Matched Sell with trade (ID: 623345521, DateTime: 2023-11-27T03:02:01-05:00)
Processing stock lot (ID: 535739081)
Matched Buy with trade (ID: 535739081, DateTime: 2023-07-10T03:02:08-04:00)
Matched Sell with trade (ID: 623345521, DateTime: 2023-11-27T03:02:01-05:00)
Processing stock lot (ID: 354534284)
Matched Buy with trade (ID: 354534284, DateTime: 2022-09-01T06:40:09-04:00)
Matched Sell with trade (ID: 623345521, DateTime: 2023-11-27T03:02:01-05:00)
Processing stock lot (ID: 354537668)
Matched Buy with trade (ID: 354537668, DateTime: 2022-09-01T07:08:48-04:00)
Matched Sell with trade (ID: 623345521, DateTime: 2023-11-27T03:02:01-05:00)
Processing stock lot (ID: 354536022)
Matched Buy with trade (ID: 354536022, DateTime: 2022-09-01T07:01:55-04:00)
Matched Sell with trade (ID: 623345521, DateTime: 2023-11-27T03:02:01-05:00)
Processing stock lot (ID: 354535608)
Matched Buy w

LookupError: 

### Derivatives Report

In [10]:
reportconfig = sir.EDavkiReportConfig(fromDate=arrow.get("2023"), toDate=arrow.get("2024"), ReportType=sir.EDavkiDocumentWorkflowType.ORIGINAL)
convertedCommonFormat = ibrkMapper.getGenericDerivativeTradeLinesFromIBRKTrades(mergedReports)

# print(convertedCommonFormat)
tradeReport = sir.EDavkiDerivativeReport(reportConfig=reportconfig, taxPayerInfo=taxPayerInfo)
envelope = tradeReport.createReportEnvelope()


test = tradeReport.convertTradesToIfiItems(convertedCommonFormat)





tradeCsv = tradeReport.generateDataFrameReport(convertedCommonFormat)
tradeCsv.to_csv(os.path.join(generatedDirectory, 'export-derivatives.csv'), index=False)



xmlReport = tradeReport.generateXmlReport(convertedCommonFormat, envelope)


etree.ElementTree(xmlReport).write(os.path.join(generatedDirectory, "D_IFI_4.xml"), xml_declaration=True, encoding='utf-8', pretty_print=True)


US0378331005
